In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('/content/train_ctrUa4K.csv')
data.head()

,post_id,timestamp,day_of_week,platform,user_id,location,language,text_content,translated_text_content,hashtags,...,comments_count,impressions,engagement_rate,brand_name,product_name,campaign_name,campaign_phase,user_past_sentiment_avg,user_engagement_growth,buzz_change_rate
0,kcqbs6hxybia,2024-12-09 11:26:15,Monday,Instagram,user_52nwb0a6,"Melbourne, Australia",pt,Just tried the Chromebook from Google. Best pu...,গুগল থেকে Chromebook ব্যবহার করে দেখুন। সর্বকা...,#Food,...,701,18991,0.19319,Google,Chromebook,BlackFriday,Launch,0.0953,-0.3672,19.1
1,vkmervg4ioos,2024-07-28 19:59:26,Sunday,Twitter,user_ucryct98,"Tokyo, Japan",ru,Just saw an ad for Microsoft Surface Laptop du...,স্প্রিংব্লাস্ট ২০২৫-এর সময় মাইক্রোসফট সারফেস ...,"#MustHave, #Food",...,359,52764,0.05086,Microsoft,Surface Laptop,PowerRelease,Post-Launch,0.1369,-0.4510,-42.6
2,memhx4o1x6yu,2024-11-23 14:00:12,Saturday,Reddit,user_7rrev126,"Beijing, China",ru,What's your opinion about Nike's Epic React? ...,নাইকির এপিক রিঅ্যাক্ট সম্পর্কে আপনার মতামত কী?...,"#Promo, #Food, #Trending",...,643,8887,0.45425,Nike,Epic React,BlackFriday,Post-Launch,0.2855,-0.4112,17.4
3,bhyo6piijqt9,2024-09-16 4:35:25,Monday,YouTube,user_4mxuq0ax,"Lagos, Nigeria",en,Bummed out with my new Diet Pepsi from Pepsi! ...,পেপসির নতুন ডায়েট পেপসি দেখে হতাশ! মান নিয়ে ...,"#Reviews, #Sustainable",...,743,6696,0.42293,Pepsi,Diet Pepsi,LaunchWave,Launch,-0.2094,-0.0167,-5.5
4,c9dkiomowakt,2024-09-05 21:03:01,Thursday,Twitter,user_l1vpox2k,"Berlin, Germany",hi,Just tried the Corolla from Toyota. Absolutely...,টয়োটার করোলাটা ট্রাই করলাম। সত্যিই খুব ভালো ল...,"#Health, #Travel",...,703,47315,0.08773,Toyota,Corolla,LocalTouchpoints,Launch,0.6867,0.0807,38.8


In [2]:
reviews = data['translated_text_content'].values[:12000]
labels  = data['sentiment_label'].values[:12000]




In [3]:
data['translated_text_content'].loc[1039]


'প্রতিযোগীর সাথে Microsoft Surface Duo এর তুলনা করা হচ্ছে। সুপারিশ করব না। #ফ্যাশন, #টেকসই আপনার প্রতিক্রিয়া জানতে আগ্রহী!'

In [4]:
data['sentiment_label'].loc[1039]

'Negative'

In [5]:
from collections import Counter

Counter(labels)

Counter({'Positive': 4839, 'Negative': 4854, 'Neutral': 2307})

In [6]:

punctuation = '!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~'

# get rid of punctuation
all_reviews = 'separator'.join(reviews)
all_reviews = all_reviews.lower()
all_text = ''.join([c for c in all_reviews if c not in punctuation])

# split by new lines and spaces
reviews_split = all_text.split('separator')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

<>:1: SyntaxWarning: invalid escape sequence '\]'
<>:1: SyntaxWarning: invalid escape sequence '\]'
/tmp/ipython-input-2748096650.py:1: SyntaxWarning: invalid escape sequence '\]'
  punctuation = '!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~'


In [7]:
# get rid of web address, twitter id, and digit
new_reviews = []
for review in reviews_split:
    review = review.split()
    new_text = []
    for word in review:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_reviews.append(new_text)

/tmp/ipython-input-2777777309.py:7: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):


In [8]:

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in new_reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])

In [9]:

# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  1176

Tokenized review: 
 [[57, 19, 519, 48, 25, 726, 78, 55, 109, 103, 1, 2, 9, 6, 5]]


In [10]:

# 2=positive, 1=neutral, 0=negative label conversion
encoded_labels = []
for label in labels:
    label = label.lower()   # normalize case
    if label == 'neutral':
        encoded_labels.append(1)
    elif label == 'negative':
        encoded_labels.append(0)
    elif label == 'positive':
        encoded_labels.append(2)
    else:
        raise ValueError(f"Unknown label: {label}")

encoded_labels = np.asarray(encoded_labels)



In [11]:

encoded_labels

array([2, 0, 0, ..., 2, 1, 0])

In [12]:

def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's
        or truncated to the input seq_length.
    '''

    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features


In [13]:
seq_length = 70

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements
assert len(features)==len(reviews_ints), "The features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches
print(features[:10,:10])

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [14]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of the resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(9600, 70) 
Validation set: 	(1200, 70) 
Test set: 		(1200, 70)


In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE the training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [16]:
# obtain one batch of training data

dataiter = iter(train_loader)

# Correct way to get next batch:
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size())   # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size())   # batch_size
print('Sample label: \n', sample_y)


Sample input size:  torch.Size([50, 70])
Sample input: 
 tensor([[  0,   0,   0,  ...,  10,   3,  11],
        [  0,   0,   0,  ...,  77, 471, 535],
        [  0,   0,   0,  ...,   9,   6,   5],
        ...,
        [  0,   0,   0,  ...,   2,  15,  22],
        [  0,   0,   0,  ...,  10,   3,  11],
        [  0,   0,   0,  ..., 103,  81,  77]])

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 2, 1, 2, 0, 0, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 2, 2, 0, 2, 2, 1, 2, 2,
        2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 2, 0, 0, 1, 0, 2, 2,
        1, 0])


In [17]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [18]:
import torch
import torch.nn as nn

class SentimentRNN(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentRNN, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            n_layers,
            batch_first=True,
            dropout=drop_prob
        )

        # Dropout
        self.dropout = nn.Dropout(0.3)

        # IMPORTANT: 3 output neurons
        self.fc = nn.Linear(hidden_dim, 3)

    def forward(self, x, hidden):

        batch_size = x.size(0)

        embeds = self.embedding(x)

        lstm_out, hidden = self.lstm(embeds, hidden)

        # Take last timestep
        out = lstm_out[:, -1, :]

        out = self.dropout(out)

        out = self.fc(out)

        # NO sigmoid / softmax here
        return out, hidden

    def init_hidden(self, batch_size):

        device = next(self.parameters()).device

        hidden = (
            torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device),
            torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        )

        return hidden


In [19]:
vocab_size = len(vocab_to_int) + 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, embedding_dim, hidden_dim, n_layers)



In [20]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [21]:
all_labels = []

for _, lbl in train_loader:
    all_labels.append(lbl)

all_labels = torch.cat(all_labels)

print("Unique labels:", torch.unique(all_labels))
print("Min label:", all_labels.min().item())
print("Max label:", all_labels.max().item())


Unique labels: tensor([0, 1, 2])
Min label: 0
Max label: 2


In [22]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch
import torch.nn as nn
import numpy as np

epochs = 20
counter = 0
print_every = 100
clip = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

net.train()

for e in range(epochs):
    for inputs, labels in train_loader:

        counter += 1

        inputs = inputs.to(device)
        labels = labels.to(device).long()   # NO subtracting now

        h = net.init_hidden(inputs.size(0))
        h = tuple(each.detach() for each in h)

        optimizer.zero_grad()

        output, h = net(inputs, h)

        loss = criterion(output, labels)

        loss.backward()

        nn.utils.clip_grad_norm_(net.parameters(), clip)

        optimizer.step()

        # Validation
        if counter % print_every == 0:
            val_losses = []
            net.eval()

            with torch.no_grad():
                for val_inputs, val_labels in valid_loader:

                    val_inputs = val_inputs.to(device)
                    val_labels = val_labels.to(device).long()

                    val_h = net.init_hidden(val_inputs.size(0))
                    val_h = tuple(each.detach() for each in val_h)

                    val_output, val_h = net(val_inputs, val_h)

                    val_loss = criterion(val_output, val_labels)
                    val_losses.append(val_loss.item())

            net.train()

            print(f"Epoch: {e+1}/{epochs} | "
                  f"Step: {counter} | "
                  f"Train Loss: {loss.item():.6f} | "
                  f"Val Loss: {np.mean(val_losses):.6f}")



Epoch: 1/20 | Step: 100 | Train Loss: 0.146012 | Val Loss: 0.187677
Epoch: 2/20 | Step: 200 | Train Loss: 0.079170 | Val Loss: 0.149702
Epoch: 2/20 | Step: 300 | Train Loss: 0.173798 | Val Loss: 0.142472
Epoch: 3/20 | Step: 400 | Train Loss: 0.046849 | Val Loss: 0.110544
Epoch: 3/20 | Step: 500 | Train Loss: 0.046400 | Val Loss: 0.108755
Epoch: 4/20 | Step: 600 | Train Loss: 0.055365 | Val Loss: 0.119864
Epoch: 4/20 | Step: 700 | Train Loss: 0.129053 | Val Loss: 0.112479
Epoch: 5/20 | Step: 800 | Train Loss: 0.250994 | Val Loss: 0.115580
Epoch: 5/20 | Step: 900 | Train Loss: 0.053908 | Val Loss: 0.107003
Epoch: 6/20 | Step: 1000 | Train Loss: 0.093342 | Val Loss: 0.107335
Epoch: 6/20 | Step: 1100 | Train Loss: 0.096287 | Val Loss: 0.107873
Epoch: 7/20 | Step: 1200 | Train Loss: 0.014825 | Val Loss: 0.112655
Epoch: 7/20 | Step: 1300 | Train Loss: 0.129795 | Val Loss: 0.114244
Epoch: 8/20 | Step: 1400 | Train Loss: 0.034706 | Val Loss: 0.121139
Epoch: 8/20 | Step: 1500 | Train Loss: 0.07

In [24]:
# Get test data loss and accuracy

test_losses = []
num_correct = 0
total = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net.eval()

with torch.no_grad():
    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device).long()

        # Initialize hidden state for each batch
        h = net.init_hidden(inputs.size(0))
        h = tuple(each.detach() for each in h)

        # Forward pass
        output, h = net(inputs, h)

        # Compute loss
        test_loss = criterion(output, labels)
        test_losses.append(test_loss.item())

        # Get predicted class (0,1,2)
        preds = torch.argmax(output, dim=1)

        # Count correct predictions
        num_correct += (preds == labels).sum().item()
        total += labels.size(0)

# --- Final Stats ---

print("Test Loss: {:.3f}".format(np.mean(test_losses)))

test_acc = num_correct / total
print("Test Accuracy: {:.3f}".format(test_acc))


Test Loss: 0.318
Test Accuracy: 0.931


In [25]:
import numpy as np
import torch
import torch.nn.functional as F

def preprocess_text(text, vocab_to_int, seq_length):

    text = text.lower()
    words = text.split()

    ints = []
    for word in words:
        if word in vocab_to_int:
            ints.append(vocab_to_int[word])
        else:
            ints.append(0)

    features = np.zeros(seq_length, dtype=int)

    if len(ints) <= seq_length:
        features[-len(ints):] = ints
    else:
        features[:] = ints[:seq_length]

    return features


In [26]:
def predict(net, text, vocab_to_int, seq_length=200):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)
    net.eval()

    features = preprocess_text(text, vocab_to_int, seq_length)

    inputs = torch.from_numpy(features).unsqueeze(0).to(device)

    h = net.init_hidden(1)
    h = tuple(each.detach() for each in h)

    with torch.no_grad():
        output, h = net(inputs, h)
        probs = F.softmax(output, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()

    return pred_class, probs.cpu().numpy()


In [48]:
label_map = {
    0: "Negative",
    1: "Neutral",
    2: "Positive"
}
text = "কিবোর্ড নিয়ে আমি ভীষন হতাশ"

pred_class, probabilities = predict(net, text, vocab_to_int)

print("Predicted Class:", pred_class)
print("Sentiment:", label_map[pred_class])
print("Probabilities:", probabilities)


Predicted Class: 0
Sentiment: Negative
Probabilities: [[0.4337275  0.15023589 0.4160366 ]]
